# 🛏️ Lightweight Product Ranking System
Returns relevant bedsheets for the query: **"light blue cotton bedsheets under 1500"**

In [ ]:
# 📘 Step 1: Imports
import json
import re
import pandas as pd
import numpy as np
from IPython.display import display, HTML

In [ ]:
# 📥 Step 2: Load product data
with open("shopping_results.json", "r", encoding="utf-8") as f:
    products = json.load(f)

In [ ]:
# 📊 Step 3: Convert to DataFrame
df = pd.DataFrame(products)
df["price_value"] = df["price"].str.replace("₹", "").str.replace(",", "").astype(float)

In [ ]:
# 🧼 Step 4: Clean & lowercase for matching
df["title_clean"] = df["title"].str.lower()
df["source"] = df["source"].str.lower()
df["rating"] = df.get("other_details", [{}]).apply(lambda x: float(x.get("Rating", 0)) if x and isinstance(x, dict) and "Rating" in x else 0.0)

In [ ]:
# ⚙️ Step 5: Define scoring function
def compute_score(row, query_keywords, brand_weight=1.2, rating_weight=1.0):
    title = row["title_clean"]
    price = row["price_value"]
    source = row["source"]
    rating = row["rating"]

    score = 0
    for word in query_keywords:
        if word in title:
            score += 1

    if price <= 1500:
        score += 1

    known_brands = ["amazon", "flipkart", "myntra", "ikea"]
    if any(b in source for b in known_brands):
        score += brand_weight

    if rating > 0:
        score += (rating / 5.0) * rating_weight

    return score

In [ ]:
# 🧠 Step 6: Apply ranking logic
query = "light blue cotton bedsheets under 1500"
keywords = ["light", "blue", "cotton", "bedsheet"]

df["score"] = df.apply(lambda row: compute_score(row, keywords), axis=1)
df_filtered = df[df["price_value"] <= 1500].sort_values("score", ascending=False)

In [ ]:
# 🎨 Step 7: Display nicely
def format_html(row):
    return f"""
    <div style="display:flex; align-items:center; border:1px solid #ddd; margin-bottom:10px; padding:10px; border-radius:10px;">
        <img src="{row['image_url']}" width="120" style="margin-right:20px; border-radius:8px;">
        <div>
            <h4 style="margin:0;">{row['title']}</h4>
            <p>Price: ₹{row['price_value']}</p>
            <p>Rating: {row['rating']}/5</p>
            <a href="{row['url']}" target="_blank">🔗 View Product</a>
        </div>
    </div>
    """

html_output = "".join(df_filtered.head(10).apply(format_html, axis=1).tolist())
display(HTML(html_output))